<h1 style="text-align:center;"><b>Developing the groundwater model</b></h1>

This notebook creates the groundwater model using FloPy as the first step in implementing our workflow. The generated folder is used by other notebooks. Note that the "**required**" settings defined here are critical for the subsequent steps.

In [1]:
import os
import numpy as np
import flopy
import shutil
import pickle

The developed groundwater model is a modified version of the Freyberg (1988) model.
Let's begin by importing input files.

In [2]:
# import modified inputs

path_to_input = os.path.join('.', 'modfied_inputs')

idomain_values = np.loadtxt(os.path.join(path_to_input, 'idomain.txt')) # original model domain
idomain_values = idomain_values.astype(np.int32) # integers

top_values = np.loadtxt(os.path.join(path_to_input, 'modf_top.txt'))    # modified topography

botm_values = np.loadtxt(os.path.join(path_to_input, 'modf_botm.txt'))  # modified bottom
botm_values = np.expand_dims(botm_values, axis=0) # convert to 3D

strt_values = np.loadtxt(os.path.join(path_to_input, 'init_head.txt'))  # modified initial heads

recharge_value = np.load(os.path.join(path_to_input, 'recharge.npy'), allow_pickle=True).item() # modified recharge values

with open(os.path.join(path_to_input, 'ghb.pkl'), 'rb') as f:
    ghb_data = pickle.load(f)    # modified GHB (General-Head Boundary)
    
with open(os.path.join(path_to_input, 'packagedata.pkl'), 'rb') as f:
    packagedata = pickle.load(f) # modified SFR (Streamflow Routing)

Following cells define functions to create the model and write input files for MODFLOW 6.

In [3]:
# define simulation

sim_name = 'downsc_freyberg'
sim_ws = os.path.join('paper_gw_model') # simulation work space. This generate a folder in the current directory.

if os.path.exists(sim_ws):
    shutil.rmtree(sim_ws)
    
sim = flopy.mf6.MFSimulation(sim_name=sim_name,
                             sim_ws=sim_ws,
                             exe_name='./mf6.exe', 
                             continue_=True)

In [4]:
# define simulation time

nper = 25
nstp = 1
tsmult = 1.0
length = [3652.5, 31.0, 29.0, 31.0, 30.0, 31.0, 30.0, 31.0, 31.0, 30.0, 31.0, 30.0, 31.0, 31.0, 28.0,
          31.0, 30.0, 31.0, 30.0, 31.0, 31.0, 30.0, 31.0, 30.0, 31.0]

perioddata_t= [(perlen, nstp, tsmult) for perlen in length]
tdis = flopy.mf6.ModflowTdis(sim,
                             nper=nper,
                              perioddata=perioddata_t,
                             time_units='days')

In [5]:
# define modelgrid (optional)

nlay = 1
nrow = 40
ncol = 20
dx = dy = 50.0
delc_values = np.array(nrow * [dy])
delr_values = np.array(ncol * [dx])

modelgrid = flopy.discretization.StructuredGrid(nlay=nlay,
                                                nrow=nrow,
                                                ncol=ncol,
                                                delr=delr_values,
                                                delc=delc_values, 
                                                top=top_values,
                                                botm=botm_values,
                                                idomain=idomain_values)

In [6]:
# define solver

ims = flopy.mf6.ModflowIms(sim,
                           print_option='SUMMARY',
                           complexity='COMPLEX')

In [7]:
# define the model

gwf = flopy.mf6.ModflowGwf(sim,
                           modelname=sim_name,
                           save_flows=True,
                           newtonoptions='newton')

In [8]:
# define discretization 

dis = flopy.mf6.ModflowGwfdis(gwf,
                              nlay=nlay,
                              nrow=nrow,
                              ncol=ncol,
                              delr=delr_values, 
                              delc=delc_values,
                              top=top_values,
                              botm=botm_values,
                              length_units='METERS',
                              idomain=idomain_values)
dis.set_all_data_external()  # required

In [9]:
# define initial head

ic = flopy.mf6.ModflowGwfic(gwf,
                            strt=strt_values)
ic.set_all_data_external()  # required

In the following cells, the adjustable parameters can be set to their initial values, which are later used during the inversion process. 
Since here a synthetic model is used, hydraulic conductivity (k) and specific yield (sy) are set to their true value for a homogenous aquifer.

In [10]:
# define hydraulic conductivity and aquifer type

k_values = np.full((nlay, nrow, ncol), 8.0) 
icelltype_values = np.full((nlay, nrow, ncol), 1, dtype=int)

npf = flopy.mf6.ModflowGwfnpf(gwf,
                              save_specific_discharge=True,
                              icelltype=icelltype_values,
                              k=k_values)
npf.set_all_data_external()  # required

In [11]:
# define storativity

ss_values = np.full((nlay, nrow, ncol), 1E-06)
sy_values = np.full((nlay, nrow, ncol), 0.25) 
iconvert_values = np.full((nlay, nrow, ncol), 1, dtype=int) 

steady_state = {0: True}
transient = {i: True for i in range(1, 25)}

sto = flopy.mf6.ModflowGwfsto(gwf, 
                              iconvert=iconvert_values, 
                              ss=ss_values,
                              sy=sy_values, 
                              steady_state=steady_state,
                              transient=transient)
sto.set_all_data_external()  # required

Over the next few cells, transient boundary conditions are defined. Instructions for adding the relevant packages are adapted from the MODFLOW 6 example documentation (https://modflow6-examples.readthedocs.io/en/master/introduction.html).

In [12]:
# define recharge (boundary condition)

rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge_value)
rch.set_all_data_external()  # required

In [13]:
# define well (boundary condition)

well_cells = [
    [0, 9, 16],
    [0, 11, 13],
    [0, 20, 14],
    [0, 26, 10],
    [0, 29, 6],
    [0, 34, 12],
    [0, 24, 4]]

well_data = {}
for i in range(25):
    data = []
    if i == 1:
        rate = -150.0
        last_val = 0.0
    elif i == 0:
        continue 
    elif i in [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]:
        rate = -300.0
        last_val = -550.0
    else:
        rate = -300.0
        last_val = 0.0
    for j, (a, b, c) in enumerate(well_cells):
        if j == len(well_cells) - 1: 
            data.append([a, b, c, last_val])
        else:
            data.append([a, b, c, rate])
    well_data[i] = data

well = flopy.mf6.ModflowGwfwel(gwf, 
                                print_flows=True, 
                                print_input=True,
                                save_flows=True,
                                maxbound =7,
                                stress_period_data=well_data)  
well.set_all_data_external()  # required

In [14]:
# define general-head boundary (a boundary condition)

ghb = flopy.mf6.ModflowGwfghb(gwf,
                              stress_period_data=ghb_data,
                              maxbound=30)
ghb.set_all_data_external()  # required

In [15]:
# define streamflow routing (boundary condition)

perioddata = [(0, 'inflow', 500.0)]
connectiondata = [[0, -1], [1, 0, -2], [2, 1, -3], [3, 2, -4], [4, 3, -5], [5, 4, -6], [6, 5, -7], [7, 6, -8], [8, 7, -9], [9, 8, -10], [10, 9, -11],
                 [11, 10, -12], [12, 11, -13], [13, 12, -14], [14, 13, -15], [15, 14, -16], [16, 15, -17], [17, 16, -18], [18, 17, -19], [19, 18, -20],
                 [20, 19, -21], [21, 20, -22], [22, 21, -23], [23, 22, -24], [24, 23, -25], [25, 24, -26], [26, 25, -27], [27, 26, -28], [28, 27, -29], 
                 [29, 28, -30], [30, 29, -31], [31, 30, -32], [32, 31, -33], [33, 32, -34], [34, 33, -35], [35, 34, -36], [36, 35, -37], [37, 36, -38],
                 [38, 37, -39], [39, 38]]

sfr = flopy.mf6.ModflowGwfsfr(gwf, 
                              boundnames=True,
                              print_input=True,
                              save_flows=True,
                              maximum_depth_change=1E-04,
                              unit_conversion=86400,
                              nreaches=40, 
                              packagedata=packagedata,
                              connectiondata=connectiondata,
                              perioddata=perioddata)
sfr.set_all_data_external(binary=False)  # required

**Note that in this cell, it is necessary to define "head_filerecord" and  its corresponding "saverecord".**

In [16]:
# define outputs 

oc = flopy.mf6.ModflowGwfoc(gwf, 
                            head_filerecord=f"{gwf.name}.hds", # required
                            budget_filerecord=f"{gwf.name}.cbc",
                            saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # required for head
                            printrecord=[("BUDGET", "ALL")]) 

**Note that in this cell, it is necessary to define observation file(s) if hydrogeological observation(s) is assimilated and/or prediction (forcast) is made in the next steps of the workflow.**

In [17]:
# define observation files

digits = 10
filename = f"{sim_name}.obs"
continuous = {'heads.csv': [('trgw-0-2-15', 'HEAD', (0, 2, 15)), ('trgw-0-2-9', 'HEAD', (0, 2, 9)), ('trgw-0-3-8', 'HEAD', (0, 3, 8)),
              ('trgw-0-9-1', 'HEAD', (0, 9, 1)), ('trgw-0-13-10', 'HEAD', (0, 13, 10)), ('trgw-0-15-16', 'HEAD', (0, 15, 16)),
              ('trgw-0-21-10', 'HEAD', (0, 21, 10)), ('trgw-0-22-15', 'HEAD', (0, 22, 15)), ('trgw-0-24-4', 'HEAD', (0, 24, 4)),
              ('trgw-0-26-6', 'HEAD', (0, 26, 6)), ('trgw-0-29-15', 'HEAD', (0, 29, 15)), ('trgw-0-33-7', 'HEAD', (0, 33, 7)),
              ('trgw-0-34-10', 'HEAD', (0, 34, 10))]}

obs_head = flopy.mf6.ModflowUtlobs(gwf, digits=digits, print_input=True, continuous=continuous) # if necessary

By runing the next step, generated files are stored in the sim_ws folder.

In [18]:
# write and run the simulation

sim.write_simulation()
success, buff = sim.run_simulation()
if not success:
    raise Exception("MODFLOW 6 did not terminate normally.")

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model downsc_freyberg...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package sto...
    writing package rcha_0...
    writing package wel_0...
    writing package ghb_0...
    writing package sfr_0...
    writing package oc...
    writing package obs_0...
FloPy is using the following executable to run the model: ..\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                   VERSION 6.3.0 release candidate 02/06/2022
                               ***DEVELOP MODE***

   MODFLOW 6 compiled Feb 06 2022 02:35:51 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.5.0
                             Build 20211109_000000

This software is preliminar